# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

2024-01-27 14:25:00.111225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
X_train_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Alley_missing,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
# Combine all features
X_train = X_train_numeric.join(X_train_cat)
X_val = X_val_numeric.join(X_val_cat)
X_test = X_test_numeric.join(X_test_cat)

As a refresher, preview the training data: 

In [5]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [6]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [7]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation= 'relu', input_shape= (296,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation= 'relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation= 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [8]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns= numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns= numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns= numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [9]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [11]:
X_train.shape

(1051, 296)

In [10]:
# Train the model
normalized_input_model.fit(X_train,
                           y_train,
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train,
                     y_train_scaled,
                     batch_size=32,
                     epochs=150,
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 0.5640 - mse: 0.5640 - val_loss: 0.2377 - val_mse: 0.2377
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2570 - mse: 0.2570 - val_loss: 0.1700 - val_mse: 0.1700
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2029 - mse: 0.2029 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1760 - mse: 0.1760 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1573 - mse: 0.1573 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1397 - mse: 0.1397 - val_loss: 0.1195 - val_mse: 0.1195
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1362 - mse: 0.1362 - val_loss: 0.1175 - val_mse: 0.1175
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1228 - m

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084


[0.00836282316595316, 0.00836282316595316]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1337 - mse: 0.1337


[0.13368044793605804, 0.13368044793605804]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [18]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
from sklearn.metrics import mean_squared_error

mean_squared_error(y_val, y_val_pred, squared=False)

9/9 [==============================] - 0s 6ms/step


28731.97176521033

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [20]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(296,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])
    
# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: 0.4565 - mse: 0.4565 - val_loss: 0.2101 - val_mse: 0.2101
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2399 - mse: 0.2399 - val_loss: 0.4023 - val_mse: 0.4023
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1913 - mse: 0.1913 - val_loss: 0.1506 - val_mse: 0.1506
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1622 - mse: 0.1622 - val_loss: 0.1459 - val_mse: 0.1459
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1408 - mse: 0.1408 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1301 - mse: 0.1301 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1156 - mse: 0.1156 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1081 - m

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0090 - mse: 0.0090


[0.009002652950584888, 0.009002652950584888]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [22]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1271 - mse: 0.1271


[0.12705297768115997, 0.12705297768115997]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [25]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(296,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 0.4884 - mse: 0.4884 - val_loss: 0.1749 - val_mse: 0.1749
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2414 - mse: 0.2414 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1902 - mse: 0.1902 - val_loss: 0.1132 - val_mse: 0.1132
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1642 - mse: 0.1642 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1424 - mse: 0.1424 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1353 - mse: 0.1353 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1263 - mse: 0.1263 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1127 - m

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0073 - mse: 0.0073


[0.007334420457482338, 0.007334420457482338]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1080 - mse: 0.1080


[0.10796644538640976, 0.10796644538640976]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [28]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer = 'rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 0.4041 - mse: 0.4041 - val_loss: 0.1640 - val_mse: 0.1640
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1666 - mse: 0.1666 - val_loss: 0.1195 - val_mse: 0.1195
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1407 - mse: 0.1407 - val_loss: 0.0843 - val_mse: 0.0843
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1090 - mse: 0.1090 - val_loss: 0.1521 - val_mse: 0.1521
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0978 - mse: 0.0978 - val_loss: 0.0821 - val_mse: 0.0821
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0729 - mse: 0.0729 - val_loss: 0.0936 - val_mse: 0.0936
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0551 - mse: 0.0551 - val_loss: 0.0861 - val_mse: 0.0861
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0567 - m

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0047 - mse: 0.0047


[0.0047304509207606316, 0.0047304509207606316]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [30]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0950 - mse: 0.0950


[0.09499718248844147, 0.09499718248844147]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [31]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 0.3417 - mse: 0.3417 - val_loss: 0.1139 - val_mse: 0.1139
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1632 - mse: 0.1632 - val_loss: 0.0923 - val_mse: 0.0923
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1396 - mse: 0.1396 - val_loss: 0.0837 - val_mse: 0.0837
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0928 - mse: 0.0928 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0723 - mse: 0.0723 - val_loss: 0.0856 - val_mse: 0.0856
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0568 - mse: 0.0568 - val_loss: 0.0874 - val_mse: 0.0874
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0461 - mse: 0.0461 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0349 - m

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 4.2762e-04 - mse: 4.2762e-04


[0.0004276186227798462, 0.0004276186227798462]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [33]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0951 - mse: 0.0951


[0.09513355046510696, 0.09513355046510696]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [39]:
model_dict = {'baseline':normalized_model, 'He Normal':he_model, 'Lecun Normal':lecun_model, 
              'RMS Prop Optimizer':rmsprop_model, 'Adam Optimizer':adam_model}
score_list =[]
for name, model in model_dict.items():
    train = model.evaluate(X_train, y_train_scaled)
    val = model.evaluate(X_val, y_val_scaled)
    score_list.append([name, train[0], train[1], val[0], val[1]])

score_df = pd.DataFrame(score_list, columns= ['Model', 'Train Loss', 'Train MSE', 'Validation Loss', 'Validation MSE'])
score_df

9/9 [==============================] - 0s 2ms/step - loss: 0.0951 - mse: 0.0951


,Model,Train Loss,Train MSE,Validation Loss,Validation MSE
0,baseline,0.008363,0.008363,0.133680,0.133680
1,He Normal,0.009003,0.009003,0.127053,0.127053
2,Lecun Normal,0.007334,0.007334,0.107966,0.107966
3,RMS Prop Optimizer,0.004730,0.004730,0.094997,0.094997
4,Adam Optimizer,0.000428,0.000428,0.095134,0.095134


In [40]:
score_df.sort_values('Validation MSE')

,Model,Train Loss,Train MSE,Validation Loss,Validation MSE
3,RMS Prop Optimizer,0.004730,0.004730,0.094997,0.094997
4,Adam Optimizer,0.000428,0.000428,0.095134,0.095134
2,Lecun Normal,0.007334,0.007334,0.107966,0.107966
1,He Normal,0.009003,0.009003,0.127053,0.127053
0,baseline,0.008363,0.008363,0.133680,0.133680


In [42]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1621 - mse: 0.1621


[0.16212640702724457, 0.16212640702724457]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [43]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
mean_squared_error(y_test, y_test_pred, squared=False)

5/5 [==============================] - 0s 3ms/step


31641.593737447965

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.